First of all, set environment variables and initialize spark context:

In [ ]:
%env SPARK_DRIVER_MEMORY=8g
%env PYSPARK_PYTHON=/usr/bin/python3.5
%env PYSPARK_DRIVER_PYTHON=/usr/bin/python3.5

from zoo.common.nncontext import *
sc = init_nncontext(init_spark_conf().setMaster("local[4]"))

# Regularization and dropout
Let's review some of the most common techniques to prevent overfitting, and let's apply them in practice to improve our movie classification model from the previous chapter. First let's prepare the data using the code from Chapter 3.5 and preprocess the data

In [ ]:
from keras.datasets import imdb
import numpy as np
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(nb_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Let's first try an easy model and try to optimize it afterwards:

In [ ]:
from zoo.pipeline.api.keras import models
from zoo.pipeline.api.keras import layers

original_model = models.Sequential()
original_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
original_model.add(layers.Dense(16, activation='relu'))
original_model.add(layers.Dense(1, activation='sigmoid'))

original_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['acc'])

original_model.fit(x_train, y_train,
                   epochs=20,
                   batch_size=512,
                   validation_data=(x_test, y_test))

Trained 512 records in 0.024455326 seconds. Throughput is 20936.135 records/second. Loss is 0.01585226.
Top1Accuracy is Accuracy(correct: 21341, count: 25000, accuracy: 0.85364)

Let's add L2 weight regularization to our movie review classification network:

In [ ]:
from zoo.pipeline.api.keras import regularizers

l2_model = models.Sequential()
l2_model.add(layers.Dense(16, W_regularizer=regularizers.l2(0.001),
                          activation='relu', input_shape=(10000,)))
l2_model.add(layers.Dense(16, W_regularizer=regularizers.l2(0.001),
                          activation='relu'))
l2_model.add(layers.Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['acc'])

l2_model.fit(x_train, y_train,
             nb_epoch=20,
             batch_size=512,
             validation_data=(x_test, y_test))

Trained 512 records in 0.024366594 seconds. Throughput is 21012.373 records/second. Loss is 0.13651785.
Top1Accuracy is Accuracy(correct: 21684, count: 25000, accuracy: 0.86736)

Let's add two Dropout layers in our IMDB network to see how well they do at reducing overfitting:

In [ ]:
dpt_model = models.Sequential()
dpt_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(16, activation='relu'))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(1, activation='sigmoid'))

dpt_model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['acc'])

dpt_model.fit(x_train, y_train,
              nb_epoch=20,
              batch_size=512,
              validation_data=(x_test, y_test))

Trained 512 records in 0.017992654 seconds. Throughput is 28456.057 records/second. Loss is 0.112769656. 
Top1Accuracy is Accuracy(correct: 21871, count: 25000, accuracy: 0.87484)